# Assignment 4 

In [1]:
import numpy as np
import sympy as sym
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt
import animatplot as amp

from textwrap import wrap
from matplotlib.animation import FuncAnimation
from numpy import exp
from scipy.constants import c, epsilon_0
from IPython.display import Math, display

from sympy import Derivative, symbols, simplify, fraction, Rational
from sympy.physics.mechanics import *
from IPython.display import Math, display

%config InlineBackend.figure_format = 'pdf'
init_vprinting()

In [2]:
N, q, m, C, eps0, omg, omg1, f1, gam1 = sym.symbols('N q m c \epsilon_0 \omega \omega_1 f_1 \gamma_1')

\newpage

## 2) Absorbtion coefficient ($\alpha$) and index of refraction ($n$) in a conductor

Let: 

> $N$ : Molecules per unit volume

> $c$ : Speed of light

> $\epsilon_0$ : Permittivity of free space

> $q$ : Charge of the electron 

> $m$ : Electron mass

> $\omega$ = Frequency of the electric field

> $\omega_j$ = Frequency of the electron 

> $f_j$ : Number of electrons with frequency $\omega_j$

> $\gamma_j$ : Damping constant in each molecule


**Absorbtion coefficient:** 

$$\alpha = 2 \kappa \approx \frac{Nq^2 \omega^2}{m \epsilon_0 c} \sum_j \frac{f_j \gamma_j}{(\omega_j^2 - \omega^2)^2 + \gamma_j^2 \omega^2}$$


**Index of refraction:**

$$n = \frac{c k}{\omega} \approx 1 + \frac{Nq^2}{2 m \epsilon_0} \sum_j \frac{f_j (\omega_j^2 - \omega^2)}{(\omega_j^2 - \omega^2)^2 + \gamma_j^2 \omega^2}$$

In [3]:
# Code versions of the functions
# ==============================
def abs_coeff1(omega, N=1, q=1, omega_1=1, m=1, epsilon=1, c=1, f_1=1, gamma_1=0.1):
    A = N*(q**2)*(omega**2) / (m*epsilon*c)
    B = (f_1*gamma_1) / ((omega_1**2 - omega**2)**2 + (gamma_1**2) * (omega**2))
    alpha = A*B
    return alpha

def n_approx(omega, N=1, q=1, omega_1=1, m=1, epsilon=1, c=1, f_1=1, gamma_1=0.1):
    A = (N*q**2) / (2*m*epsilon)
    B = f_1 * (omega_1**2 - omega**2) / ((omega_1**2 - omega**2)**2 + (gamma_1**2) * (omega**2))
    n = 1 + A*B
    return n

### Absorbtion coefficient:

Let's look at the absorbtion coefficient first. 

For materials with a single natural frequency $\omega_1$:
$$\alpha \approx \frac{Nq^2 \omega^2}{m \epsilon_0 c} \frac{f_1 \gamma_1}{(\omega_1^2 - \omega^2)^2 + \gamma_1^2 \omega^2}$$

#### Finding $\omega$ for maximum $\alpha$:

By observation, we can see that $\alpha$ is maximum when $\omega = \omega_1$:
$$\alpha_{Max} \approx \frac{Nq^2}{m \epsilon_0 c} \frac{f_1 \gamma_1}{\gamma_1^2}$$ 

We can see that this is the case by setting everything to 1 and $\gamma_1 = 0.1$ (because we want $\alpha_{Max}$ to be finite), and plotting $\alpha$ with respect to $\omega$:

In [4]:
omega_space = np.arange(0.001, 2, 0.001)

fig = plt.figure(1, figsize=(10, 4))

plt.plot(omega_space, abs_coeff1(omega=omega_space, gamma_1=0.1), label=(r"$\alpha$ $\left[ \frac{q^2 \omega^2}{m \epsilon_0 c} \right]$"))
plt.plot(omega_space, n_approx(omega=omega_space, gamma_1=0.1), label=(r"$n$ $\left[ \frac{q^2}{m \epsilon_0 c} \right]$"))
plt.xlabel(r"$\omega$")
title1 = r"Relationship between absorbtion coefficient of a conductor ($\alpha$) and index of refraction ($n$), and incident frequency of light ($\omega$) with electron damping ($\gamma = 0.1$)"
plt.title("\n".join(wrap(title1, 100)))
plt.legend()
plt.grid()
plt.show()

<Figure size 720x288 with 1 Axes>

For a more rigorous proof, let's do this analytically:

Let's take the derivative of $\alpha$:

In [5]:
alpha_sym = abs_coeff1(omg, N, q, omg1, m, eps0, C, f1, gam1)
d_alpha_sym = sym.diff(alpha_sym, omg).simplify()
display(Math(r'\frac{\text{d} \alpha}{\text{d} \omega} =' + sym.latex(d_alpha_sym)))

<IPython.core.display.Math object>

Setting this to zero, we can obtain the values at which $\alpha$ is maximum:

In [6]:
omg_max = sym.solve(d_alpha_sym, omg)
display(Math(r'\omega_{Max} =' + sym.latex(omg_max)))

<IPython.core.display.Math object>

As frequency must be real and non-negative, we have $\omega = \omega_1$ which is thet same as what we observed. Substituting this into our equation for $\alpha$, this gives us an expression for $\alpha_{Max}$:

In [7]:
alpha_max = alpha_sym.subs(omg, omg1)
display(Math(r'\alpha_{Max} =' + sym.latex(alpha_max)))

<IPython.core.display.Math object>

#### Finding $\omega$ for $\frac{1}{2} \alpha_{Max}$:

To do this, we set $\alpha = \frac{1}{2} \alpha_{Max}$ then solve for $\omega$ 

In [8]:
omg_half = sym.solve(alpha_max - alpha_sym/2, omg)
omg_half = [i.simplify() for i in omg_half]
display(Math(r'\omega_{1/2} =' + sym.latex(omg_half)))

<IPython.core.display.Math object>

### Index of refraction:

#### Frequencies ($\omega$) for maximum and minimum index of refraction ($n$):

Take the derivative of $n$:

In [14]:
n_sym = n_approx(omg, N, q, omg1, m, eps0, C, f1, gam1)
d_n = sym.diff(n_sym, omg).simplify()
dd_n =  sym.diff(n_sym, omg, 2).simplify()
display(Math(r'n =' + sym.latex(n_sym)))
display(Math(r'\frac{\text{d} n}{\text{d} \omega} =' + sym.latex(d_n)))
display(Math(r'\frac{\text{d}^2 n}{\text{d}^2 \omega} =' + sym.latex(dd_n)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [16]:
omg_ex = sym.solve(d_n, omg)
display(Math(r'\omega_{Extremum} =' + sym.latex(omg_ex)))

<IPython.core.display.Math object>

Here's our values of $\omega$ for the extremums of $n$, but we don't know which ones are the minimum and which ones are the maximum. 

Let's substitute these values into $\frac{\text{d}^2 n}{\text{d}^2 \omega}$:

In [23]:
display(Math(r'\left.\frac{\text{d}^2 n}{\text{d}^2 \omega} \right|_{\omega_{Extremum}} =' + sym.latex([dd_n.subs(omg, omg_ex[i]).simplify() for i in range(len(omg_ex))])))

<IPython.core.display.Math object>

Assuming that $\omega_1 >> \gamma_1$, we can see:

> 1st term: Negative

> 2nd term: Negative

> 3rd term: Negative 

> 4th term: Positive

> 5th term: Positve 


This tells us that values of $\omega$ for maximum $n$: 

> $\omega_{Max_n} = \sqrt{\gamma_1 \omega_1 + \omega_1^2}$

Values of $\omega$ for minimum $n$: 

> $\omega_{Min_n} = \sqrt{-\gamma_1 \omega_1 + \omega_1^2}$

Here we have ignored $0$ and the negative terms of $\omega_{Extremum}$ because frequency must be positive. 

\newpage

## 3) Transverse electric waves in a rectangular waveguide

A rectangular waveguide with side lengths $0\leq x \leq a$ and $0 \leq y \leq b$.

The boundary conditions in a rectangular waveguide: 

$$\left\{ \begin{array}{ll}
    E^{\parallel} = 0\\
    B^{\perp} = 0
    \end{array} \right.$$  
    
The electric and magnetic fields have the generic form:

$$\left\{ \begin{array}{ll}
    E(x, y, z, t) = E_0(x, y) e^{i(kz-\omega t)}\\
    B(x, y, z, t) = B_0(x, y) e^{i(kz-\omega t)}
    \end{array} \right.$$ 
    
Following Griffiths, applying Maxwells equation to these and their results we (actually, just him) obtain the differential equations:

$$\left\{ \begin{array}{ll}
    \left[ \frac{\partial^2}{\partial^2 x} + \frac{\partial^2}{\partial^2 y} + \left(\frac{\omega}{c}\right)^2 - k^2 \right] E_z = 0\\
    \left[ \frac{\partial^2}{\partial^2 x} + \frac{\partial^2}{\partial^2 y} + \left(\frac{\omega}{c}\right)^2 - k^2 \right] B_z = 0
    \end{array} \right.$$ 

### a) Solving for TM waves:

To solve for TM waves, we set $B_z = 0$. 

Then we use separation of variables: 

$$E_z(x, y) = X(x)Y(y)$$

Substituting this into our differential equation for $E_z$ we obtain: 

$$Y \frac{\text{d}^2X}{\text{d}^2x} + X \frac{\text{d}^2Y}{\text{d}^2y} + \left[\left(\frac{\omega}{c}\right)^2 - k^2 \right] XY = 0$$

Dividing this by $XY$, we obtain: 

$$\left\{ \begin{array}{ll}
    \frac{1}{X} \frac{\text{d}^2X}{\text{d}^2x} = -k_x^2 \\
    \frac{1}{Y} \frac{\text{d}^2Y}{\text{d}^2y} = -k_y^2 \\
    -k_x^2 - k_y^2 + \left(\frac{\omega}{c}\right)^2 - k^2 = 0
    \end{array} \right.$$ 
    
We know the general solution (from math and more importantly, Griffiths) that the general solution: 

$$\left\{ \begin{array}{ll}
    X(x) = A \sin(k_x x) + B\cos(k_x x) \\
    Y(y) = D \sin(k_y y) + E\cos(k_y y)
    \end{array} \right.$$ 
    
    
The boundary conditions require $E^\parallel = 0$, therefore: 

$$\left\{ \begin{array}{ll}
    X(0) = A \sin(0) + B\cos(0) = 0 \\
    X(a) = A \sin(k_x a) + B\cos(k_x a) = 0 \\
    Y(0) = D \sin(0) + E\cos(0) = 0 \\
    Y(b) = D \sin(k_y m) + E\cos(k_y m) = 0
    \end{array} \right.$$ 

This gives us: 

$$\left\{ \begin{array}{ll}
    B = E = 0\\
    k_x = \frac{n \pi}{a} \\
    k_y = \frac{m \pi}{b}
    \end{array} \right.$$
    
$$\text{Where:  } (n, m) \in (0 < \mathbb{Z})$$

The equation to the TM wave:

$$\left\{ \begin{array}{ll}
    E_z(x, y) = E_0 \sin \left(\frac{n \pi}{a} x \right)\sin \left(\frac{m \pi}{b} y\right)\\
    k = \left[ \left(\frac{\omega}{c}\right)^2  - \left( \frac{n \pi}{a} \right)^2 - \left(\frac{m \pi}{b}\right)^2 \right]^{\frac{1}{2}}
    \end{array} \right.$$

### b) Poynting vector for TM waves:

Poynting vector: 

$$\vec{S} = \frac{1}{\mu_0} Re\left( \vec{E} \times \vec{B^*} \right)$$

\newpage

## 4) COMPUTATION: Finite Difference Time Domain method

In [9]:
def E(t, E=1, s=1, t0=0):
    E_field = E * exp(-(t - t0)**2 / s**2)
    return E_field